In [1]:
import pandas
import tensorflow
import datetime as dt
from pathlib import Path
import os

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


print(pandas.__version__)
print(tensorflow.__version__)

1.3.5
2.8.2


# 1.Data Preprocessing

### README

In [2]:
import markdown
readme = '/content/drive/MyDrive/Colab Notebooks/LMS/predict_moive/README'
f = open(readme, 'r')
htmlmarkdown=markdown.markdown( f.read() )
htmlmarkdown

'<h1>SUMMARY</h1>\n<p>These files contain 1,000,209 anonymous ratings of approximately 3,900 movies \nmade by 6,040 MovieLens users who joined MovieLens in 2000.</p>\n<h1>USAGE LICENSE</h1>\n<p>Neither the University of Minnesota nor any of the researchers\ninvolved can guarantee the correctness of the data, its suitability\nfor any particular purpose, or the validity of results based on the\nuse of the data set.  The data set may be used for any research\npurposes under the following conditions:</p>\n<pre><code> * The user may not state or imply any endorsement from the\n   University of Minnesota or the GroupLens Research Group.\n\n * The user must acknowledge the use of the data set in\n   publications resulting from the use of the data set\n   (see below for citation information).\n\n * The user may not redistribute the data without separate\n   permission.\n\n * The user may not use this information for any commercial or\n   revenue-bearing purposes without first obtaining permiss

결과물이 만족스럽진 않다. html로 쓰여져 있어 가독성이 많이 떨어짐

In [3]:
# md파일을 읽는 함수
import markdown
from bs4 import BeautifulSoup
def read_readme(path):
    f = open(path,'r')
    md_to_html = markdown.markdown( f.read() )
    soup = BeautifulSoup(md_to_html)
    print(soup.get_text())

In [4]:
read_readme(readme)

SUMMARY
These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 
made by 6,040 MovieLens users who joined MovieLens in 2000.
USAGE LICENSE
Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under the following conditions:
 * The user may not state or imply any endorsement from the
   University of Minnesota or the GroupLens Research Group.

 * The user must acknowledge the use of the data set in
   publications resulting from the use of the data set
   (see below for citation information).

 * The user may not redistribute the data without separate
   permission.

 * The user may not use this information for any commercial or
   revenue-bearing purposes without first obtaining permission
   from a faculty member of the GroupLens Research Project a

USERS FILE DESCRIPTION

User data set 정보

형식은 다음과 같음

UserID | Gender | Age | Occupation | Zip-code

user ID| 성별 | 나이 | 직종 | 우편번호


이 정도면 이름 빼고 대부분의 정보를 기록해놨다.

## 1.1.1 Data Load

In [5]:
data_path = Path('/content/drive/MyDrive/Colab Notebooks/LMS/predict_moive') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    data.sort_values(['UserId', 'Time'] ,inplace=True)
    return data

data = load_data(train_path, None)
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


10,000,209개 의 데이터가 있다. 약 천만개 정도의 분량

In [6]:
data['UserId'].nunique(), data['ItemId'].nunique()

(6040, 3706)

유저 id가 6040개 , 영화 갯수가 3706이라 생각

## 1.1.2 Other Data Set

### Movies.dat

In [7]:
movies_path = data_path / 'movies.dat'
users_path = data_path / 'users.dat'

def load_data2(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], nrows=nrows, encoding='cp1252')
    data.columns = ['ItemId', 'Title', 'Genre']

    return data

movie_data = load_data2(movies_path, None)

In [8]:
movie_data

,ItemId,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


??? 위의 ratings 데이터 셋에서는 3706개의 유니크한 데이터가 있었지만, 여긴 3883개의 영화에 대한 정보가 들어있다.

### Users.dat

In [9]:
def load_data_user(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3,4], nrows=nrows, encoding='cp1252')
    data.columns = ['UserID ','Gender', 'Age ' ,'Occupation' , 'Zip-code']

    return data

In [10]:
user_data = load_data_user(users_path, None)
user_data

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


## 1.2 Session Length ( SessionId를 공유하는 데이터 row의 개수 )

## 유저 id를 기준으로 각 세션이 얼마나 있는지 알아내야 한다.

## 해야 할 일

> 1. 유저 id별로 datetiem을 묶는다.
> 2. 세션 id를 생성한다. 
> 3. 같은 세션 id끼리 묶어서 세션 길이를 구한다. 

아래 블로그에서 time columns에 있는 의미 불명의 숫자를 datetime형식으로 바꿔주는 코드를 발견했다.

https://velog.io/@cha-suyeon/Python-timestamp-%ED%98%95%EC%8B%9D%EC%9D%84-datetime-%ED%98%95%EC%8B%9D%EC%9C%BC%EB%A1%9C-%EB%B0%94%EA%BF%94%EC%A3%BC%EB%8A%94-%EB%B0%A9%EB%B2%95

epoch timestamp 를 dateime으로 변환해주는 추가적인 정보

https://www.javatpoint.com/python-epoch-to-datetime

In [11]:
from datetime import datetime
# import time

date = []

for i in range(1000209) :
    # time = datetime.fromtimestamp(data['Time'][i]).strftime('%Y-%m-%d %H:%M:%S')
    time = datetime.fromtimestamp(data['Time'][i])
    date.append(time)


data['date'] = date

In [12]:
oldest, latest = data['date'].min(), data['date'].max()
print(oldest) 
print(latest)

2000-04-25 23:05:32
2003-02-28 17:49:50


보기 편하도록 time column 제거

In [13]:
data = data.drop('Time',axis=1)

In [14]:
data.sort_values(['UserId', 'date'], inplace=True)
data.reset_index(drop=True, inplace=True)
data

,UserId,ItemId,Rating,date
0,1,2918,4,2000-12-31 22:00:19
1,1,150,5,2000-12-31 22:00:55
2,1,2797,4,2000-12-31 22:00:55
3,1,1197,3,2000-12-31 22:00:55
4,1,1097,4,2000-12-31 22:01:43
...,...,...,...,...
1000204,6040,2286,1,2001-08-10 14:40:29
1000205,6040,2150,3,2001-08-10 14:41:04
1000206,6040,1900,5,2001-08-10 14:41:04
1000207,6040,1249,4,2001-08-10 14:41:26


### 세션 ID 생성

세션 Id는 User ID와 묶여 있어야 하며, 브라우저 종료시 같이 종료되어야 한다.

In [15]:
UserId_length = data.groupby('UserId').size()
UserId_length

UserId
1        53
2       129
3        51
4        21
5       198
       ... 
6036    888
6037    202
6038     20
6039    123
6040    341
Length: 6040, dtype: int64

In [16]:
Example = UserId_length.index[0]
Example
data[data['UserId']==Example]

,UserId,ItemId,Rating,date
0,1,2918,4,2000-12-31 22:00:19
1,1,150,5,2000-12-31 22:00:55
2,1,2797,4,2000-12-31 22:00:55
3,1,1197,3,2000-12-31 22:00:55
4,1,1097,4,2000-12-31 22:01:43
5,1,1029,5,2000-12-31 22:02:52
6,1,1022,5,2000-12-31 22:04:35
7,1,2804,5,2000-12-31 22:11:59
8,1,2355,5,2000-12-31 22:11:59
9,1,3186,4,2000-12-31 22:12:40


세션은 시간 데이터를 기준으로 생성

In [17]:
date_copy = data.copy()
# date_copy['Time_diff'] = date_copy['date'].diff().fillna(0)
# date_copy['Change_User'] = date_copy['UserId'].diff().fillna(0)

data_ex = date_copy['date'].diff()

date_copy['timedelta'] = data_ex
date_copy['timedelta'][0] = date_copy['timedelta'][2] # 0번째 인덱스가 NaT이므로 0과 같은 값으로 결측치를 채운다.
user_diff = date_copy['UserId'].diff().fillna(0)
date_copy['User_diff'] = user_diff
date_copy.isnull().sum()

UserId       0
ItemId       0
Rating       0
date         0
timedelta    0
User_diff    0
dtype: int64

일반적으로 세션의 만료시간이 15~20분 이므로 timedelta = 15 이상인 행들만 찾는다.

In [18]:
import datetime
def decide_session_len (df,minute = 59, second = 59):
    # 일정 시간 이상 'timedelta'컬럼에서 액션의 시간차가 발생하는 데이터 프레임 
    lenth = df[(df['User_diff']!=1) & (df['timedelta'] > datetime.timedelta(hours = 0 , minutes =minute, seconds =second))]
    return lenth

In [19]:
len = decide_session_len(date_copy,15,0)

In [20]:
len.size

123630

약 12만3600개의 데이터가 포함됨

코드 참고
https://github.com/hjkornn-phys/Aiffel-Exploration-projects/blob/master/Exploration_12/EX12_Session_based_Recommendation.ipynb

In [21]:
session_id = 1
def make_sessions(df):
    global session_id
    if df['timedelta'] > datetime.timedelta(hours = 0 , minutes =15, seconds =0) or df['User_diff']==1:
        session_id += 1
    return session_id

In [22]:
data['SessionId'] = date_copy.apply(make_sessions, axis=1)

In [23]:
data

,UserId,ItemId,Rating,date,SessionId
0,1,2918,4,2000-12-31 22:00:19,1
1,1,150,5,2000-12-31 22:00:55,1
2,1,2797,4,2000-12-31 22:00:55,1
3,1,1197,3,2000-12-31 22:00:55,1
4,1,1097,4,2000-12-31 22:01:43,1
...,...,...,...,...,...
1000204,6040,2286,1,2001-08-10 14:40:29,26644
1000205,6040,2150,3,2001-08-10 14:41:04,26644
1000206,6040,1900,5,2001-08-10 14:41:04,26644
1000207,6040,1249,4,2001-08-10 14:41:26,26644


In [24]:
session_length = data.groupby('SessionId').size()
session_length

SessionId
1         40
2         13
3        129
4         51
5         21
        ... 
26641      7
26642     14
26643      1
26644     21
26645      1
Length: 26645, dtype: int64

In [25]:
# 한 세션의 길이가 3이상인 세션을 고른다.
effective_sessionid = session_length.apply(lambda g: g>=3)
effective_sessionid

SessionId
1         True
2         True
3         True
4         True
5         True
         ...  
26641     True
26642     True
26643    False
26644     True
26645    False
Length: 26645, dtype: bool

### 세션 ID별로 묶기

In [26]:
# 모든 value가 True 인 Series
sessionids = effective_sessionid[effective_sessionid==True]

In [27]:
session_list = sessionids.index.to_list()
type(session_list)
# len(session_list)
#분명 list 타입인데...

list

## 1.4 Data Cleansing


In [28]:
clean_data = data[data['SessionId'].isin(session_list)]
clean_data

,UserId,ItemId,Rating,date,SessionId
0,1,2918,4,2000-12-31 22:00:19,1
1,1,150,5,2000-12-31 22:00:55,1
2,1,2797,4,2000-12-31 22:00:55,1
3,1,1197,3,2000-12-31 22:00:55,1
4,1,1097,4,2000-12-31 22:01:43,1
...,...,...,...,...,...
1000203,6040,589,4,2001-08-10 14:39:58,26644
1000204,6040,2286,1,2001-08-10 14:40:29,26644
1000205,6040,2150,3,2001-08-10 14:41:04,26644
1000206,6040,1900,5,2001-08-10 14:41:04,26644


In [29]:
clean_data = clean_data[['UserId', 'ItemId', 'Rating', 'date', 'SessionId']]
clean_data

,UserId,ItemId,Rating,date,SessionId
0,1,2918,4,2000-12-31 22:00:19,1
1,1,150,5,2000-12-31 22:00:55,1
2,1,2797,4,2000-12-31 22:00:55,1
3,1,1197,3,2000-12-31 22:00:55,1
4,1,1097,4,2000-12-31 22:01:43,1
...,...,...,...,...,...
1000203,6040,589,4,2001-08-10 14:39:58,26644
1000204,6040,2286,1,2001-08-10 14:40:29,26644
1000205,6040,2150,3,2001-08-10 14:41:04,26644
1000206,6040,1900,5,2001-08-10 14:41:04,26644


In [30]:
oldest, latest = clean_data['date'].min(), clean_data['date'].max()
session_length = clean_data.groupby('SessionId').size()

In [31]:
session_length.median(), session_length.mean()

(27.0, 59.50557396806267)

In [32]:
session_length.min(), session_length.max()

(3, 1193)

In [33]:
session_length.quantile(0.999)

792.8120000000054

In [34]:
length_count = session_length.groupby(session_length).size()
length_percent_cumsum = length_count.cumsum() / length_count.sum()
length_percent_cumsum_999 = length_percent_cumsum[length_percent_cumsum < 0.999]

length_percent_cumsum_999

3      0.081952
4      0.134980
5      0.180838
6      0.216270
7      0.245014
         ...   
768    0.998735
769    0.998795
771    0.998855
777    0.998915
792    0.998976
Length: 522, dtype: float64

In [35]:
cumsum_95 = length_percent_cumsum_999[length_percent_cumsum_999<0.95]
cumsum_95

3      0.081952
4      0.134980
5      0.180838
6      0.216270
7      0.245014
         ...   
225    0.948177
226    0.948358
227    0.948719
228    0.948961
229    0.949684
Length: 227, dtype: float64

In [36]:
effective_sessionid = session_length.apply(lambda g: 3<=g<=229)
print(effective_sessionid)
print(effective_sessionid.mean())

SessionId
1        True
2        True
3        True
4        True
5        True
         ... 
26639    True
26640    True
26641    True
26642    True
26644    True
Length: 16595, dtype: bool
0.9496836396504972


In [37]:
clean_data = clean_data[clean_data['SessionId'].isin(session_list)]
clean_data.reset_index(drop=True, inplace=True)
clean_data

,UserId,ItemId,Rating,date,SessionId
0,1,2918,4,2000-12-31 22:00:19,1
1,1,150,5,2000-12-31 22:00:55,1
2,1,2797,4,2000-12-31 22:00:55,1
3,1,1197,3,2000-12-31 22:00:55,1
4,1,1097,4,2000-12-31 22:01:43,1
...,...,...,...,...,...
987490,6040,589,4,2001-08-10 14:39:58,26644
987491,6040,2286,1,2001-08-10 14:40:29,26644
987492,6040,2150,3,2001-08-10 14:41:04,26644
987493,6040,1900,5,2001-08-10 14:41:04,26644


## 1.5 Train / Valid / Test split

In [38]:
start_timestamp = latest - dt.timedelta(30*20)     # 최종 날짜로부터 17개월 이전 날짜를 구한다.  
datasets = clean_data[clean_data['date'] > start_timestamp]   # 방금 구한 날짜 이후의 데이터만 모은다.
datasets

,UserId,ItemId,Rating,date,SessionId
2497,20,3578,5,2001-12-29 23:37:51,47
2498,20,1923,4,2001-12-29 23:37:51,47
2499,20,1694,3,2001-12-29 23:37:51,47
2500,20,2641,4,2001-12-29 23:37:51,47
2501,20,3863,3,2001-12-29 23:38:35,47
...,...,...,...,...,...
987490,6040,589,4,2001-08-10 14:39:58,26644
987491,6040,2286,1,2001-08-10 14:40:29,26644
987492,6040,2150,3,2001-08-10 14:41:04,26644
987493,6040,1900,5,2001-08-10 14:41:04,26644


In [39]:
# 점수가 3 이상인 것만 남김
datasets = datasets[datasets['Rating']>=3]

In [40]:
datasets

,UserId,ItemId,Rating,date,SessionId
2497,20,3578,5,2001-12-29 23:37:51,47
2498,20,1923,4,2001-12-29 23:37:51,47
2499,20,1694,3,2001-12-29 23:37:51,47
2500,20,2641,4,2001-12-29 23:37:51,47
2501,20,3863,3,2001-12-29 23:38:35,47
...,...,...,...,...,...
987489,6040,2664,4,2001-08-10 14:39:27,26644
987490,6040,589,4,2001-08-10 14:39:58,26644
987492,6040,2150,3,2001-08-10 14:41:04,26644
987493,6040,1900,5,2001-08-10 14:41:04,26644


In [41]:
# short_session을 제거한 다음 unpopular item을 제거하면 다시 길이가 1인 session이 생길 수 있습니다.
# 이를 위해 반복문을 통해 지속적으로 제거 합니다.
def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
    while True:
        before_len = data.shape[0]
        data = cleanse_short_session(data, shortest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = data.shape[0]
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby('SessionId').size()
    session_use = session_len[session_len >= shortest].index
    data = data[data['SessionId'].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby('ItemId').size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data['ItemId'].isin(item_use)]
    return data

In [42]:
# least_click의 경우 rating의 최소 개수를 의미합니다.
cleansed_datasets = cleanse_recursive(datasets, shortest=3, least_click=20)
cleansed_datasets

,UserId,ItemId,Rating,date,SessionId
2497,20,3578,5,2001-12-29 23:37:51,47
2498,20,1923,4,2001-12-29 23:37:51,47
2500,20,2641,4,2001-12-29 23:37:51,47
2501,20,3863,3,2001-12-29 23:38:35,47
2502,20,457,4,2001-12-29 23:38:35,47
...,...,...,...,...,...
987485,6040,562,5,2001-08-10 14:36:00,26644
987488,6040,1278,4,2001-08-10 14:39:09,26644
987490,6040,589,4,2001-08-10 14:39:58,26644
987492,6040,2150,3,2001-08-10 14:41:04,26644


In [43]:
data = data.sort_values('date')
data.head(10)

,UserId,ItemId,Rating,date,SessionId
999868,6040,2303,5,2000-04-25 23:05:32,26637
999869,6040,573,4,2000-04-25 23:05:54,26637
999870,6040,1264,4,2000-04-25 23:05:54,26637
999871,6040,592,2,2000-04-25 23:06:17,26637
999872,6040,1270,3,2000-04-25 23:06:17,26637
999873,6040,858,4,2000-04-25 23:07:36,26637
999874,6040,3505,4,2000-04-25 23:07:36,26637
999875,6040,162,4,2000-04-25 23:07:36,26637
999876,6040,1975,1,2000-04-25 23:07:36,26637
999877,6040,2987,1,2000-04-25 23:07:36,26637


In [44]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['date'].max()
    session_last_time = data.groupby('date')['date'].max()
    session_in_train = session_last_time[session_last_time < final_time - dt.timedelta(n_days)].index
    session_in_test = session_last_time[session_last_time >= final_time - dt.timedelta(n_days)].index

    before_date = data[data['date'].isin(session_in_train)]
    after_date = data[data['date'].isin(session_in_test)]
    
    return before_date, after_date

In [45]:
tr, test = split_by_date(data, n_days=180)
tr, val = split_by_date(tr, n_days=180)

In [46]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [47]:
save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

tr.to_pickle(save_path / 'train.pkl')
val.to_pickle(save_path / 'valid.pkl')
test.to_pickle(save_path / 'test.pkl')

# 2.DataPipeLine

## 2.1 SessionDataset

In [48]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['SessionId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['SessionId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('SessionId').size().cumsum()
        return offsets

In [49]:
tr_dataset = SessionDataset(tr)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,date,SessionId,item_idx
999868,6040,2303,5,2000-04-25 23:05:32,26637,0
999869,6040,573,4,2000-04-25 23:05:54,26637,1
999870,6040,1264,4,2000-04-25 23:05:54,26637,2
999871,6040,592,2,2000-04-25 23:06:17,26637,3
999872,6040,1270,3,2000-04-25 23:06:17,26637,4
999873,6040,858,4,2000-04-25 23:07:36,26637,5
999874,6040,3505,4,2000-04-25 23:07:36,26637,6
999875,6040,162,4,2000-04-25 23:07:36,26637,7
999876,6040,1975,1,2000-04-25 23:07:36,26637,8
999877,6040,2987,1,2000-04-25 23:07:36,26637,9


In [50]:
tr_dataset.click_offsets

array([     0,     40,     53, ..., 979072, 979093, 979094], dtype=int32)

In [51]:
tr_dataset.session_idx

array([    0,     1,     2, ..., 23222, 23223, 23224])

## 2.2 SessionDataLoader

In [52]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += mask.shape[0]  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

In [53]:
tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(15)

,UserId,ItemId,Rating,date,SessionId,item_idx
999868,6040,2303,5,2000-04-25 23:05:32,26637,0
999869,6040,573,4,2000-04-25 23:05:54,26637,1
999870,6040,1264,4,2000-04-25 23:05:54,26637,2
999871,6040,592,2,2000-04-25 23:06:17,26637,3
999872,6040,1270,3,2000-04-25 23:06:17,26637,4
999873,6040,858,4,2000-04-25 23:07:36,26637,5
999874,6040,3505,4,2000-04-25 23:07:36,26637,6
999875,6040,162,4,2000-04-25 23:07:36,26637,7
999876,6040,1975,1,2000-04-25 23:07:36,26637,8
999877,6040,2987,1,2000-04-25 23:07:36,26637,9


In [54]:
iter_ex = iter(tr_data_loader)

In [55]:
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [  0  40  53 175]
Label Item Idx are :       [  1  41  54 108]
Previous Masked Input Idx are []


# 3. Modeling

## 3.1 Evaluation Metric

In [56]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

## 3.2 Model Architecture

In [57]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [58]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [59]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['SessionId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

args = Args(tr, val, test, batch_size=2048, hsz=50, drop_rate=0.1, lr=0.001, epochs=3, k=20)

In [60]:
args.tr.shape[0]

979094

In [61]:
model = create_model(args)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(2048, 1, 3705)]         0         
                                                                 
 GRU (GRU)                   [(2048, 50),              563550    
                              (2048, 50)]                        
                                                                 
 dropout (Dropout)           (2048, 50)                0         
                                                                 
 dense (Dense)               (2048, 3705)              188955    
                                                                 
Total params: 752,505
Trainable params: 752,505
Non-trainable params: 0
_________________________________________________________________


## 3.3 Model Training

In [62]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = args.tr.shape[0] - args.tr['SessionId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = data.shape[0] - data['SessionId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = inputs.shape[0]
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [63]:
# 학습 시간이 다소 오래 소요됩니다. 아래 주석을 풀지 마세요.
train_model(model, args)

Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]


IndexError: ignored

# 진짜 너무 화가 난다.

In [ ]:
# 학습된 모델을 불러옵니다.
model = tf.keras.models.load_model(data_path / 'trained_model')

## 3.4 Inference

In [ ]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)

# 회고

일단 데이터 전처리에서 너무 에러가 많이 나서 화가 많이나고, 분명 LMS나 다른 사람들 코드에서는 잘만 돌아가다 내가 할 때는 에러가 발생해서 화가 좀 많이 났다.


#이리 저리 에러를 해결해보려 했지만 되지 않는다,어떻게 하라는건지 모르겠다.

혼자 힘으로 해보려고 했는데 잘 되지도 않고 얻어가는게 없어서 의욕만 떨어졌다.